# Colorizing Grayscale Images Using Neural Networks

## Import the necessary libraries

In [ ]:
import os
import tensorflow as tf
import keras
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from skimage.transform import resize
import numpy as np
import random
from PIL import Image
import requests

## Load in our training data

We will specifically focus on colorizing images of landscapes.  First, let's focus on training our model on only one image.

In [38]:
url = 'https://assets.website-files.com/5d315dd990eed30aa7ca5f8f/5d3d843acb7dfd63bcf37e29_tree-and-woodland-company-landscape.jpg'

In [39]:
image = Image.open(requests.get(url, stream=True).raw)
image.show()

## Preprocessing the input image

In [40]:
image = keras.preprocessing.image.img_to_array(image)
image = np.array(image, dtype=float)
image.shape

(1350, 2400, 3)

In [41]:
input = rgb2lab(1.0/255*image)[:,:,0]
input = resize(input, (400, 400))
input = input.reshape((1, 400, 400, 1))

In [42]:
output = rgb2lab(1.0/255*image)[:,:,1:]
output /= 128
output = resize(output, (400, 400, 2))
output = output.reshape((1, 400, 400, 2))

## Build the Neural Network

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same')
])

## Compile the model

In [45]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

## Train the model

In [46]:
model.fit(x=input, y=output, batch_size=1, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.3206 - accuracy: 0.6197
Epoch 2/100
1/1 [==============================] - 0s 4ms/step - loss: 0.2967 - accuracy: 0.5490
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.0431 - accuracy: 0.5372
Epoch 4/100
1/1 [==============================] - 0s 998us/step - loss: 0.0332 - accuracy: 0.5358
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0316 - accuracy: 0.5292
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0334 - accuracy: 0.5483
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0316 - accuracy: 0.5746
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0506 - accuracy: 0.5489
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 0.0349 - accuracy: 0.4906
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0310 - accuracy: 0.5477
Epoch 11/100
1/1 [=========

## Evaluate the model

In [47]:
model.evaluate(input, output, batch_size=1, verbose=2)

1/1 - 0s - loss: 0.0069 - accuracy: 0.8990


[0.00685169966891408, 0.8990437388420105]

## Colorizing images using the model

Let's colorize the image we used to train the model on.

In [49]:
result = model.predict(input)
result *= 128
colorized_img = np.zeros((400, 400, 3))
colorized_img[:,:,0] = input[0][:,:,0]
colorized_img[:,:,1:] = result[0]
imsave("original_img.png", resize(image, (400, 400, 3)))
imsave("colorized_img.png", lab2rgb(colorized_img))
imsave("grayscale_img.png", rgb2gray(lab2rgb(colorized_img)))

Lossy conversion from float64 to uint8. Range [0.18796588134546266, 212.59868580095446]. Convert image to uint8 prior to saving to suppress this warning.
C:\Program Files\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\skimage\color\colorconv.py:1128: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Even though our model was able to colorize the image pretty well, it is still a weak neural network. To understand this, try colorizing images the model has not been trained on.

In [53]:
url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_kjXxkqXog81CD2olSDquVyi92iu-GVCH0w&usqp=CAU'

In [54]:
image = Image.open(requests.get(url, stream=True).raw)
image = keras.preprocessing.image.img_to_array(image)
image = np.array(image, dtype=float)

In [55]:
input = rgb2lab(1.0/255*image)[:,:,0]
input = resize(input, (400, 400))
input = input.reshape(1, 400, 400, 1)

In [56]:
result = model.predict(input)
result *= 128
colorized_img = np.zeros((400, 400, 3))
colorized_img[:,:,0] = input[0][:,:,0]
colorized_img[:,:,1:] = result[0]
imsave("original_img.png", resize(image, (400, 400, 3)))
imsave("colorized_img.png", lab2rgb(colorized_img))
imsave("grayscale_img.png", rgb2gray(lab2rgb(colorized_img)))


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


We will use a Kaggle dataset of [landscape pictures]('https://www.kaggle.com/arnaud58/landscape-pictures') for our training images.

In [ ]:
img_dataset = []
